<a href="https://colab.research.google.com/github/tecton-jon/onboarding-materials/blob/main/Tecton_Rift_Lab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 🧪 Lab: Productionizing Real-Time Features with Tecton and Rift

In this lab, we will explore how we can develop and test real-time features for a fraud detecton use case using Tecton and Rift.

Rift is Tecton's Python-first compute engine for efficiently computing batch, stream, and real-time features using Python and SQL. With Rift we can develop and test features locally in any Python environment and then productionize with a single step.

Let's try it out!

## ⚙️ Install Pre-Reqs

Run the following commands to install Tecton and other pre-requisites.

**After installation, be sure to restart your session via "Runtime -> Restart Session" in the menu above.**

In [ ]:
!pip install virtualenv
!virtualenv tecton
!source tecton/bin/activate
!pip install 'tecton[rift]' s3fs fsspec scikit-learn

✅ Restart your session via "Runtime -> Restart Session"

## 👩‍💻 Log into a Tecton account

In [ ]:
import tecton, os, requests, json
import pandas as pd
from pprint import pprint
from tecton import *
from tecton.types import *
from datetime import datetime, timedelta

tecton.set_validation_mode('auto')

tecton.login('lab.tecton.ai')

## 🔎 Examine Raw Data

On S3 we have a historical log of a transaction stream representing transactions that users made at different merchants in the last few years.

We can use this data to brainstorm streaming features and even test them out with Tecton!

In [ ]:
df = pd.read_parquet("s3://tecton.ai.public/tutorials/fraud_demo/transactions/data.pq", storage_options={'anon': True})

display(df)

## 🌊 Define and Test Streaming Features

Streaming features can be tested offline in a notebook and used to train a model. Tecton uses the historical log of a stream to compute accurate historical feature values.

✅ Try extending the definition below with more features, such as:

- The total dollar amount of transactions a user has made in the last 1 minute, 1 hour, and 1 year.
- The total number of transactions a user has made in the last 1 minute, 1 hour, and 1 year.

You may find [this documentation](https://docs.tecton.ai/docs/beta/defining-features/feature-views/aggregation-engine/aggregation-functions) helpful.

In [ ]:
# Define a stream source, including the historical log of the stream
transactions_stream = StreamSource(
    name='transactions_stream',
    stream_config=PushConfig(),
    batch_config=FileConfig(
        uri='s3://tecton.ai.public/tutorials/fraud_demo/transactions/data.pq',
        file_format='parquet',
        timestamp_field='timestamp'
    ),
    schema=[Field('user_id', String), Field('timestamp', Timestamp), Field('amt', Float64)]
)

# Define the entity we are creating features for
user = Entity(name='user', join_keys=['user_id'])

# Define features
@stream_feature_view(
    source=transactions_stream,
    entities=[user],
    mode='pandas',
    aggregations=[
        Aggregation(function='mean', column='amt', time_window=timedelta(minutes=1)),
        Aggregation(function='mean', column='amt', time_window=timedelta(minutes=5)),
        Aggregation(function='mean', column='amt', time_window=timedelta(days=365))
    ],
    schema=[Field("user_id", String), Field("timestamp", Timestamp), Field("amt", Float64)]
)
def user_transaction_features(transactions):
    return transactions[['user_id', 'timestamp', 'amt']]


# Compute features
start = datetime(2023,1,1)
end = datetime(2023,6,1)

feature_df = user_transaction_features.get_historical_features(start_time=start, end_time=end).to_pandas()

display(feature_df)

## ⏱️ Define and Test Real-Time Features

Now let's define define a feature that checks if the current transaction amount a user is seeking to make is higher than their historical average.

Because this feature depends on real-time info (the current transaction amount), we need to compute it at the time of the request. That's exactly where on-demand features come in.

✅ Try changing the definition below to compare the transaction to the 1 year average instead of the 5 minute average.

In [ ]:
# Define on-demand features
transaction_request = RequestSource(schema=[Field("amt", Float64)])

@on_demand_feature_view(
    sources=[transaction_request, user_transaction_features],
    mode="python",
    schema=[Field("transaction_amount_is_higher_than_average", Bool)],
)
def transaction_amount_is_higher_than_average(transaction_request, user_transaction_features):
    amount_mean = user_transaction_features["amt_mean_5m_continuous"]
    amount_mean = 0 if amount_mean is None else amount_mean
    return {"transaction_amount_is_higher_than_average": transaction_request["amt"] > amount_mean}


# Test on-demand features
averages = feature_df.drop(columns=['user_id', 'timestamp', '_effective_timestamp']).iloc[0].to_dict()
request = {'amt': 10.4}
features = transaction_amount_is_higher_than_average.run(user_transaction_features=averages, transaction_request=request)

print('\nRequest amount: ' + str(request['amt']))
print('Average: ' + str(averages['amt_mean_5m_continuous']))
print(str(features))

## 🧮 Generate Training Data

Now that we've created some features, it's time to join them into a training data set so we can train a model.

First let's load up a list of historical training events. These events represent labeled historical user transactions.

In [ ]:
training_events = pd.read_parquet("s3://tecton.ai.public/tutorials/fraud_demo/transactions/data.pq", storage_options={'anon': True}) \
                    [['user_id', 'timestamp', 'amt', 'is_fraud']]

display(training_events)

Now that we have our training events, we can get features for those events by adding them to a Feature Service and calling `get_historical_features(events)`.

The feature service defines the set of features we want to serve to our model offline and online.

In [ ]:
from tecton import FeatureService

fraud_detection_feature_service = FeatureService(
    name="fraud_detection_feature_service",
    features=[user_transaction_features, transaction_amount_is_higher_than_average]
)

training_data = fraud_detection_feature_service.get_historical_features(training_events).to_pandas()

display(training_data)

## 🧠 Train a Model

With a training dataset full of features, we can now train a simple logistic regression model to detect fraudulent transactions.

In [ ]:
from sklearn.pipeline import make_pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn import metrics

df = training_data.drop(['user_id', 'timestamp', 'amt'], axis=1)

X = df.drop('is_fraud', axis=1)
y = df['is_fraud']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

num_cols = X_train.select_dtypes(exclude=['object']).columns.tolist()
cat_cols = X_train.select_dtypes(include=['object']).columns.tolist()

num_pipe = make_pipeline(
    SimpleImputer(strategy='median'),
    StandardScaler()
)

cat_pipe = make_pipeline(
    SimpleImputer(strategy='constant', fill_value='N/A'),
    OneHotEncoder(handle_unknown='ignore', sparse_output=False)
)

full_pipe = ColumnTransformer([
    ('num', num_pipe, num_cols),
    ('cat', cat_pipe, cat_cols)
])

model = make_pipeline(full_pipe, LogisticRegression(max_iter=1000, random_state=42))

model.fit(X_train,y_train)

y_predict = model.predict(X_test)

print(model)

## 🚀 Apply Features to Production

**NOTE: This step has been done for you already.**

Productionizing features with Tecton is easy. Simply paste the definitions into a repo of Python files, select a workspace, and run `tecton apply to productize

Create a feature repo:
```bash
mkdir feature-repo && cd feature-repo
tecton init
touch features.py
```

Apply features to production:
```bash
tecton login lab.tecton.ai
tecton workspace select prod
tecton apply
```

You can check out the applied features in Tecton's web UI [here](https://lab.tecton.ai/app/repo/prod/features).


## ⚡️ Ingest Streaming Events and Read Real-Time Features

Once we've productionized our Stream Source, we can start sending events to it. Any features defined against this source will be updated in real time!

Try adding your own name as the `user_id` below and watch how feature values update immediately.

In [ ]:
tecton.set_credentials(tecton_api_key='3fcbbfb66c3c4d1a7ce1b9e02f410a1f')
os.environ['TECTON_API_KEY'] = '3fcbbfb66c3c4d1a7ce1b9e02f410a1f'

In [ ]:
ws = tecton.get_workspace('prod')
registered_transactions_stream = ws.get_data_source('transactions_stream')

In [ ]:
registered_transactions_stream.ingest({
    'user_id': 'matt-lab',
    'timestamp': datetime.utcnow(),
    'amt': 100.00
})

In [ ]:
fs = ws.get_feature_service('fraud_detection_feature_service')
features = fs.get_online_features(join_keys={'user_id': 'matt-lab'}, request_data={'amt': 50}).to_dict()

pprint(features)

## 🔥 Define Online Prediction Pipeline

Now that we have online feature values, we can create a prediction pipeline to determine if a transaction is fraudulent and whether we should accept or reject it.

To do this we will define three functions to:

1. Get features from Tecton
2. Use the real-time features to make a prediction with the model
3. Use the model prediction to accept or reject a transaction

In [ ]:
# Get features from Tecton
def get_online_feature_data(user_id, amt):
    headers = {"Authorization": "Tecton-key " + os.environ['TECTON_API_KEY']}

    request_data = f'''{{
        "params": {{
            "feature_service_name": "fraud_detection_feature_service",
            "join_key_map": {{"user_id": "{user_id}"}},
            "metadata_options": {{"include_names": true}},
            "request_context_map": {{"amt": {amt}}},
            "workspace_name": "prod"
        }}
    }}'''

    online_feature_data = requests.request(
        method="POST",
        headers=headers,
        url="https://lab.tecton.ai/api/v1/feature-service/get-features",
        data=request_data,
    )

    online_feature_data_json = json.loads(online_feature_data.text)

    return online_feature_data_json

# Use the real-time features to make a prediction with the model
def get_prediction_from_model(feature_data):
    columns = [f["name"].replace(".", "__") for f in feature_data["metadata"]["features"]]
    data = [feature_data["result"]["features"]]

    features = pd.DataFrame(data, columns=columns)

    return model.predict(features)[0]

# Use the model prediction to accept or reject a transaction
def evaluate_transaction(user_id, amt):
    online_feature_data = get_online_feature_data(user_id, amt)
    is_predicted_fraud = get_prediction_from_model(online_feature_data)

    print('Features: ' + str(online_feature_data["result"]["features"]))
    print('Model Score: ' + str(is_predicted_fraud))

    if is_predicted_fraud == 0:
        print('Transaction accepted.')
    else:
        print('Transaction denied.')

## ⭐️ Evaluate Transactions in Real-Time

Now we have a single decision API to evaluate transactions in real-time!

Let's test it out.

In [ ]:
evaluate_transaction('matt', 182.46)